In [1]:
%%capture
!pip install wandb --upgrade

In [2]:
# Import wandb
import wandb

# Login to W&B
wandb.login()

wandb: Currently logged in as: sxs200126 (use `wandb login --relogin` to force relogin)


True

In [3]:
pip install torch-lr-finder

In [4]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary
import albumentations as A
import tensorflow as tf
import tarfile
from fastai.vision import *
from torchvision import models, transforms, datasets

from collections import defaultdict

from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
#from torch_lr_finder import LRFinder

import os

from shutil import copy
from shutil import copytree, rmtree

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import random

from datetime import datetime
from pathlib import Path
import plotly.io as pio
pio.renderers.default = 'colab'
from torchvision import models

In [5]:
# Import random function
import random

# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# specify the folder in googe drive where we will save dataset
# this is a string
basepath = '/content/drive/MyDrive/Data'

In [8]:
# specify pathlib folder
# This is a system Path(PosixPath)
folder = Path(basepath)
folder

PosixPath('/content/drive/MyDrive/Data')

In [9]:
file = folder / 'food-101.tgz'

In [10]:
#with tarfile.open(file, 'r' )as tar:
#                  tar.extractall(path=folder)



In [11]:
#for entries in folder.iterdir():
#  print(entries.name)

In [12]:
#for entries in (folder/'food-101').iterdir():
#  print(entries.name)

In [13]:
# Check the extracted dataset folder
#os.listdir('food-101/')

In [14]:
#os.listdir('food-101/images')

In [15]:
#os.listdir('food-101/meta')

# Split the image data into train and test using train.txt and test.txt

In [16]:
# Helper method to split dataset into train and test folders
#def prepare_data(filepath, src,dest):
#  classes_images = defaultdict(list)
#  with open(filepath, 'r') as txt:
#      paths = [read.strip() for read in txt.readlines()]
#      for p in paths:
#        food = p.split('/')
#        classes_images[food[0]].append(food[1] + '.jpg')

#  for food in classes_images.keys():
#    print("\nCopying images into ",food)
#    if not os.path.exists(os.path.join(dest,food)):
#      os.makedirs(os.path.join(dest,food))
#    for i in classes_images[food]:
#      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
#  print("Copying Done!")

In [17]:
# Prepare train dataset by copying images from food-101/images to food-101/train using the file train.txt
#print("Creating train data...")
#prepare_data(folder/'food-101/train.txt', folder/'food-101/images', folder/'food-101/train')

In [18]:
# Prepare test data by copying images from food-101/images to food-101/test using the file test.txt
#print("Creating test data...")
#prepare_data(folder/'food-101/test.txt', folder/'food-101/images', folder/'food-101/test')

In [19]:
# Check how many files are in the train folder

train_files = sum([len(files) for i, j, files in os.walk(folder/"food-101/train")])
print("Total number of samples in train folder")
print(train_files)

Total number of samples in train folder
75750


In [20]:
# Check how many files are in the test folder
test_files = sum([len(files) for i, j, files in os.walk(folder/"food-101/test")])
print("Total number of samples in test folder")
print(test_files)

Total number of samples in test folder
25250


## **Data Download and Transform to tensor**

In [21]:
class FOOD101():
    def __init__(self):
        self.train_ds, self.test_ds, self.train_cls, self.test_cls = [None]*4
        self.imgenet_mean = imagenet_stats[0]
        self.imgenet_std = imagenet_stats[1]
   
        
    def _get_tfms(self):
        train_tfms = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(), 
            transforms.ToTensor(),
            transforms.Normalize(self.imgenet_mean, self.imgenet_std)])
        
        test_tfms = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(self.imgenet_mean, self.imgenet_std)])        
        return train_tfms, test_tfms            
            
    def get_dataset(self,root_dir=folder/'food-101'):
        train_tfms, test_tfms = self._get_tfms() # transformations
        self.train_ds = datasets.ImageFolder(root=folder/"food-101/train", transform=train_tfms)
        self.test_ds = datasets.ImageFolder(root=folder/"food-101/test", transform=test_tfms)        
        self.train_classes = self.train_ds.classes
        self.test_classes = self.test_ds.classes

        assert self.train_classes==self.test_classes
        return self.train_ds, self.test_ds, self.train_classes

    
    
food = FOOD101()    

In [22]:
train_ds, test_ds, classes =  food.get_dataset()
num_classes = len(classes)
print(classes)
print(num_classes)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [23]:
train_ds, test_ds

(Dataset ImageFolder
     Number of datapoints: 75750
     Root location: /content/drive/MyDrive/Data/food-101/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ), Dataset ImageFolder
     Number of datapoints: 25250
     Root location: /content/drive/MyDrive/Data/food-101/test
     StandardTransform
 Transform: Compose(
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ))

In [24]:
trainset, validset = torch.utils.data.random_split(train_ds, [60000, 15750], generator=torch.Generator().manual_seed(42) )
testset = test_ds

In [25]:
check_loader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle = True)

In [26]:
len(check_loader)

1875

In [27]:
len(check_loader.dataset)

60000

In [28]:
trainset.dataset

Dataset ImageFolder
    Number of datapoints: 75750
    Root location: /content/drive/MyDrive/Data/food-101/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [29]:
train_sample_size = int(len(trainset)/100)
valid_sample_size = int(len(validset)/50)
test_sample_size = int(len(testset)/50)
# Getting n random indices
train_subset_indices = random.sample(range(0, len(trainset)), train_sample_size)
valid_subset_indices = random.sample(range(0, len(validset)), valid_sample_size)
test_subset_indices = random.sample(range(0, len(testset)), test_sample_size)

# Getting subset of dataset
train_subset = torch.utils.data.Subset(trainset, train_subset_indices)
valid_subset = torch.utils.data.Subset(validset, valid_subset_indices)
test_subset = torch.utils.data.Subset(testset, test_subset_indices)

In [30]:
# load model with random weights
model = models.resnet101(pretrained=True)

In [31]:
#print(model)

# Training Functions

## Training Epoch 

In [32]:
def train(train_loader, model, optimizer, loss_function, log_batch, log_interval, grad_clipping, max_norm):

  """ 
  Function for training the model in each epoch
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate.
  Output: final weights, bias, train loss, train accuracy
  """
  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_train
  global batch_ct_train

  # Training Loop loop
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode
  model.train()

  # Iterate on batches from the dataset using train_loader
  for input, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input = input.to(device)
    targets = targets.to(device)

    # Forward pass
    output = model(input)
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    example_ct_train +=  len(targets)
    batch_ct_train += 1

    # set gradients to zero 
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Update parameters using their gradient
    optimizer.step()

    #scheduler.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})
        #print(f'Learning rate: {scheduler.get_last_lr()}')

    
    
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)



  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)

  return train_loss, train_acc

## Validation/Test Epoch

In [33]:
def valid(loader, model, optimizer, loss_function, log_batch, log_interval):

  """ 
  Function for training the model and plotting the graph for train & valid loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and valid loss for each epoch.
  """

  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_valid
  global batch_ct_valid

  # Validation loop
  # Initialize train_loss at the he strat of the epoch
  running_valid_loss = 0
  running_valid_correct = 0
  
  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input,targets in loader:

      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)

      # Forward pass
      output = model(input)
      loss = loss_function(output,targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      # count of images and batches
      example_ct_valid +=  len(targets)
      batch_ct_valid += 1

      # Add valid loss of a batch 
      running_valid_loss += loss.item()

      # Add correct count for each batch
      running_valid_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})


    # Calculate mean valid loss for the whole dataset for a particular epoch
    valid_loss = running_valid_loss/len(valid_loader)

    # scheduler step
    # scheduler.step(valid_loss)
    # scheduler.step()

    # Calculate accuracy for the whole dataset for a particular epoch
    valid_acc = running_valid_correct/len(valid_loader.dataset)
    
  return valid_loss, valid_acc

In [34]:
def train_loop(train_loader, valid_loader, model, loss_function, optimizer, epochs, device, patience, early_stopping,
               file_model):

  '''
  model: specify your model for training
  criterion: loss function 
  optimizer: optimizer like SGD , ADAM etc.
  train loader: function to carete batches for training data
  loader : function to create batches for valid data set
  file_model : specify file name for saving your model. This way we can upload the model weights from file. We will not to run model again.
  

  '''
  # Create lists to store train and valid loss at each epoch

  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []
  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False


  # Iterate for the given number of epochs
  for epoch in range(epochs):
    t0 = datetime.now()
    # Get train loss and accuracy for one epoch

    train_loss, train_acc = train(train_loader, model, optimizer, loss_function, 
                                  wandb.config.log_batch, wandb.config.log_interval,
                                  wandb.config.grad_clipping, wandb.config.max_norm)
    valid_loss, valid_acc = valid(valid_loader, model, optimizer, loss_function,
                                    wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    else:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss



    # Log the train and valid loss to W&B
    wandb.log({f"Train epoch Loss :": train_loss, f"Valid epoch Loss :": valid_loss })
    wandb.log({f"Train epoch Acc :": train_acc, f"Valid epoch Acc :": valid_acc})



    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler.get_last_lr()}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history


# Model Training

In [35]:
hyperparameters = dict(
    epochs = 10,
    output_dim = 10, 
    
    batch_size = 128,
    learning_rate = 0.001,
    dataset="food_101",
    architecture="resnet101",
    log_interval = 100,
    log_batch = True,
    file_model = folder/'homework6B_resnet101.pt',
    grad_clipping = False,
    max_norm = 1,
    patience = 5,
    early_stopping = False,
    weight_decay = 0,
    scheduler_factor = 0.5,
    scheduler_patience = 0,
   )

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [36]:
device

device(type='cpu')

In [37]:
wandb.init(name = 'Shishir_hw6b_resnet101_task', project = 'homework6b_DL_food_101', config = hyperparameters)

In [38]:
wandb.config.device = device
print(wandb.config.device )

cpu


In [39]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=wandb.config.batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_subset, batch_size=wandb.config.batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_subset, batch_size=wandb.config.batch_size,   shuffle = False)



# cross entropy loss function
loss_function = nn.CrossEntropyLoss()

# Model 
# Task 1
#vgg16_task1 = models.vgg16(pretrained=True)
#vgg16_task1.classifier[6].out_features = wandb.config.output_dim

# freeze convolution weights
#for param in vgg16_task1.features.parameters():
#    param.requires_grad = False

#model = vgg16_task1 

# Task 2
#vgg16_task2 = models.vgg16(pretrained=True)
#vgg16_task2.classifier[6].out_features = wandb.config.output_dim

# freeze convolution weights
#for param in vgg16_task2.features[:24].parameters():
#    param.requires_grad = False

#model = vgg16_task2 

# Task 3
#vgg16_task3 = models.vgg16(pretrained=True)
#vgg16_task3.classifier[6].out_features = wandb.config.output_dim

# freeze convolution weights
#for param in vgg16_task3.features[:17].parameters():
#    param.requires_grad = False
#model = vgg16_task3 

# task 4

#resnet50_task4 = models.resnet50(pretrained=True)
#resnet50_task4.fc.out_features = wandb.config.output_dim

# Freeze all but last CNN block
#count = 0
#for child in resnet50_task4.children():
#  count+=1
#  if count < 8:
#    for param in child.parameters():
#        param.requires_grad = False

# task 4

resnet101_task4 = models.resnet101(pretrained=True)
resnet101_task4.fc.out_features = wandb.config.output_dim

# Freeze all but last CNN block
count = 0
for child in resnet101_task4.children():
  count+=1
  if count < 8:
    for param in child.parameters():
        param.requires_grad = False

model = resnet101_task4


def init_weights(m):
  if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

  if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

        
# apply initialization recursively  to all modules
# model.apply(init_weights)

#wandb.config.init_weights = init_weights

# put model to GPUs
model.to(wandb.config.device)

# Intialize stochiastic gradient descent optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay, momentum = 0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay)

wandb.config.optimizer = optimizer

#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
#                              patience=wandb.config.scheduler_patience, verbose=True)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 1000, eta_min=0, last_epoch=- 1, verbose=False)
#
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.2, total_steps=len(train_loader) * 20 , 
#                                                epochs=5, three_phase=True, pct_start = 0.3)

#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.2, total_steps=len(train_loader) * 30 , 
#                                                epochs=30, three_phase=True, pct_start = 0.2)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

## Sanity Check
- Check the loss without any training. For Cross entropy the expected value will be log(number of classes)

In [40]:
# Fix seed value

SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

for input, targets in train_loader:
  
  # move inputs and outputs to GPUs
  input = input.to(device)
  targets = targets.to(device)
  model.eval()
  # Forward pass
  output = model(input)
  loss = loss_function(output, targets)
  print(f'Actual loss: {loss}')
  break

print(f'Expected Theoretical loss: {np.log(10)}')



Actual loss: 14.858500480651855
Expected Theoretical loss: 2.302585092994046


## Train Model and Save best model

In [41]:
wandb.watch(model, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [42]:
example_ct_train, batch_ct_train, example_ct_valid, batch_ct_valid = 0, 0, 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, valid_loader, model, loss_function, optimizer, 
                                                                                          wandb.config.epochs, wandb.config.device,
                                                                                          wandb.config.patience, wandb.config.early_stopping,
                                                                                          wandb.config.file_model)

Validation loss has decreased (inf --> 11.559856). Saving Model...
Epoch : 1 / 10
Time to complete 1 is 0:05:49.160197
Train Loss:  9.4114 | Train Accuracy:  0.0000%
Valid Loss:  11.5599 | Valid Accuracy:  0.0000%

Validation loss has decreased (11.559856 --> 9.799099). Saving model...
Epoch : 2 / 10
Time to complete 2 is 0:05:42.158075
Train Loss:  8.9098 | Train Accuracy:  0.0000%
Valid Loss:  9.7991 | Valid Accuracy:  0.0000%

Validation loss has decreased (9.799099 --> 8.816882). Saving model...
Epoch : 3 / 10
Time to complete 3 is 0:05:42.040226
Train Loss:  8.4161 | Train Accuracy:  0.3333%
Valid Loss:  8.8169 | Valid Accuracy:  0.3175%

Validation loss has decreased (8.816882 --> 8.135810). Saving model...
Epoch : 4 / 10
Time to complete 4 is 0:05:42.671031
Train Loss:  7.8014 | Train Accuracy:  0.8333%
Valid Loss:  8.1358 | Valid Accuracy:  0.3175%

Validation loss has decreased (8.135810 --> 7.592007). Saving model...
Epoch : 5 / 10
Time to complete 5 is 0:05:41.677648
Train L